In [1]:
import torch
import torch.nn.functional as F

In [2]:
# need 1.10 for cross_entropy using probabilities for each class
torch.__version__

'1.10.2+cu113'

In [3]:
from math import exp, log, log2, log10
import numpy as np

In [4]:
input_shape = (4,3,2,2)

In [5]:
input_array = torch.rand(input_shape)

In [6]:
input_array.shape

torch.Size([4, 3, 2, 2])

In [7]:
input_array[0]

tensor([[[0.8255, 0.2463],
         [0.0967, 0.2857]],

        [[0.8029, 0.7693],
         [0.8196, 0.8302]],

        [[0.0506, 0.8951],
         [0.0170, 0.9357]]])

In [8]:
target_array = torch.rand(input_shape)

In [9]:
target_array[0]

tensor([[[0.2759, 0.2657],
         [0.4227, 0.3155]],

        [[0.3252, 0.7790],
         [0.3257, 0.2497]],

        [[0.6062, 0.4456],
         [0.9601, 0.1985]]])

## MSELoss

In [9]:
mse_loss = MSELoss(reduction='none')

In [10]:
output = mse_loss(sample, target)

In [11]:
output = torch.sum(output, dim=1)

In [12]:
output.shape

torch.Size([3, 2, 2])

### confirm output loss is correct

In [13]:
output[0][0][0]

tensor(0.8281)

In [14]:
sample[0]

tensor([[[0.5877, 0.2064],
         [0.6624, 0.0816]],

        [[0.9403, 0.6024],
         [0.3144, 0.9906]],

        [[0.7961, 0.0529],
         [0.4201, 0.1197]]])

In [15]:
target[0]

tensor([[[0.8001, 0.6994],
         [0.1338, 0.9786]],

        [[0.0906, 0.9563],
         [0.3613, 0.3918]],

        [[0.5490, 0.9035],
         [0.0273, 0.7414]]])

In [16]:
test_array = []
for i in range(3):
    loss = (sample[0][i][0][0] - target[0][i][0][0])**2
    test_array.append(loss)

In [17]:
# across 3 classes
sum(test_array)

tensor(0.8281)

## Cross Entropy Loss

In [10]:
input_array[0]

tensor([[[0.8255, 0.2463],
         [0.0967, 0.2857]],

        [[0.8029, 0.7693],
         [0.8196, 0.8302]],

        [[0.0506, 0.8951],
         [0.0170, 0.9357]]])

In [12]:
input_array.shape

torch.Size([4, 3, 2, 2])

In [11]:
target_array[0]

tensor([[[0.2759, 0.2657],
         [0.4227, 0.3155]],

        [[0.3252, 0.7790],
         [0.3257, 0.2497]],

        [[0.6062, 0.4456],
         [0.9601, 0.1985]]])

In [14]:
target_array.shape

torch.Size([4, 3, 2, 2])

In [15]:
output = F.cross_entropy(input_array, target_array, reduction='none')

In [16]:
output.shape

torch.Size([4, 2, 2])

In [17]:
output[0]

tensor([[1.5531, 1.5779],
        [2.2026, 0.9070]])

In [18]:
output[1]

tensor([[1.8507, 0.7137],
        [2.2131, 0.9993]])

### Verifying calculation

In [19]:
denominator = input_array.clone()
denominator = torch.exp(denominator)
denominator = denominator.sum(axis=1, keepdim=True)

In [20]:
denominator.shape

torch.Size([4, 1, 2, 2])

In [21]:
consistency_loss = input_array.clone()
consistency_loss = - torch.log(torch.exp(consistency_loss) / denominator) * target_array
consistency_loss = consistency_loss.sum(axis=1)

In [22]:
consistency_loss[0]

tensor([[1.5531, 1.5779],
        [2.2026, 0.9070]])

In [23]:
consistency_loss[1]

tensor([[1.8507, 0.7137],
        [2.2131, 0.9993]])

In [24]:
np.allclose(output, consistency_loss)

True

## Inverse exponential loss

In [25]:
output

tensor([[[1.5531, 1.5779],
         [2.2026, 0.9070]],

        [[1.8507, 0.7137],
         [2.2131, 0.9993]],

        [[1.9240, 1.3868],
         [1.3340, 1.3083]],

        [[0.6362, 0.2954],
         [1.6721, 2.0073]]])

In [26]:
weights = output.clone()

In [27]:
weights.apply_(lambda x: 1/exp(abs(x)))

tensor([[[0.2116, 0.2064],
         [0.1105, 0.4038]],

        [[0.1571, 0.4898],
         [0.1094, 0.3682]],

        [[0.1460, 0.2499],
         [0.2634, 0.2703]],

        [[0.5293, 0.7442],
         [0.1879, 0.1344]]])

In [24]:
# examine high weight
sample[2]

tensor([[[0.7524, 0.3060],
         [0.1306, 0.0195]],

        [[0.6729, 0.6879],
         [0.3878, 0.0731]],

        [[0.7534, 0.6265],
         [0.9868, 0.6129]]])

In [26]:
target[2]

tensor([[[0.9832, 0.7142],
         [0.7276, 0.5473]],

        [[0.5862, 0.2266],
         [0.2031, 0.8077]],

        [[0.7425, 0.5329],
         [0.2231, 0.0180]]])